In [1]:
from openai import OpenAI
import os
import base64
import requests
import json
import requests
import pandas as pd
import sys
#Get the current working directory
notebook_dir = os.getcwd()

# Add the root directory to the Python path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

# Import the API key from config
from config.config import API_KEY


In [2]:
def gpt_authenticate():
    try:
        API_key = API_KEY
        client = OpenAI(api_key=API_key)
        
        # Test the client with a simple API call
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hello"}],
            max_tokens=5
        )
        print("Authentication successful!")
        return client
        
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        return None


In [7]:
def check_image_url(url):
    try:
        # Send a GET request to the image URL
        response = requests.get(url)
        
        # Check if the response status code is 200 (OK)
        if response.status_code == 200:
            # Get the Content-Type header from the response
            content_type = response.headers.get('Content-Type', '')
            
            # Check if the Content-Type is an image format
            if 'image' in content_type:
                print(f"Success: URL {url} is accessible and is of type {content_type}.")
                return False  # No error
            else:
                print(f"Error: URL {url} is not an image. Content-Type: {content_type}")
                return True  # Error occurred
        else:
            print(f"Error: URL {url} returned status code {response.status_code}.")
            return True  # Error occurred
    
    except requests.exceptions.RequestException as e:
        print(f"Error: Failed to access URL {url}. Exception: {e}")
        return True  # Error occurred

In [8]:
#%% using links as images
def gpt_extract(client, url):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": '''Describe the image (Figure/Graphical abstract) from an article on comorbidity between COVID-19 and Neurodegeneration.   
                                      1. Name potential mechanisms (pathophysiological processes) of Covid-19's impact on the brain depicted in the image. 
                                      2. Describe each process depicted in the image as semantic triples (subject–predicate–object).  
                                      Example: 
                                      Pathophysiological Process: Astrocyte_Activation 
                                      Triples:
                                      SARS-CoV-2_infection|triggers|astrocyte_activation
                                      
                                      Use ONLY the information shown in the image! Follow the structure precisely and don't write anything else! Replace spaces in names with _ sign, make sure that words "Pathophysiology Process:" and "Triples:" are presented, don't use bold font and margins. Each triple must contain ONLY THREE elements separated by a | sign, four and more are not allowed!'''},
          {
            "type": "image_url",
            "image_url": {
              "url": url,
            },
          },
        ],
      }
    ],
    max_tokens=900,
  )
    content = response.choices[0].message.content
    return content

In [ ]:
path = 'C:\\Users\\User\\Desktop\\Data\\SCAI\\'
relevance_GPT = pd.read_excel(path+"FINAL_RELEVANT_URLs.xlsx")
relevance_GPT.drop(['Unnamed: 0', 'Relevance_manual'], axis=1, inplace=True)
relevance_GPT

In [ ]:
def triples_extraction_from_urls(path_URLs):

    client = gpt_authenticate(API_key)
    relevance_GPT = pd.read_excel(path_URLs)

    # Initialize an empty list to store parsed data
    parsed_data = []

    # Loop through each row in the dataframe
    for idx, row in (relevance_GPT.head(5)).iterrows():
        try:
            url = row["URL"]  # Extract image URL

            # Extract content from the image using GPT
            content = gpt_extract(client, url)
            print(url, content)

            # Parse the text for mechanisms and triples
            mechanisms = content.strip().split('Pathophysiology Process: ')

            for mechanism_block in mechanisms[1:]:
                lines = mechanism_block.strip().split('\n')
                mechanism_name = lines[0].strip()
                triples = lines[2:]  # Skip the 'Triples:' line

                for triple in triples:
                    subject, action, obj = triple.strip().split('|')
                    parsed_data.append([url, mechanism_name, subject, action, obj])

        except Exception as e:
            # Print the error and continue to the next row
            print(f"Error processing {url}: {e}")
            continue  # Skip to the next row in case of an error

    # Create a new DataFrame to store parsed results
    parsed_df = pd.DataFrame(parsed_data, columns=['URL', 'Pathophysiological Process', 'Subject', 'Predicate', 'Object'])

    # Save dataframe
    parsed_df.to_csv('Triples_Final.csv')
    parsed_df.to_excel(r"Triples_Final.xlsx", index=False)
    print('Triples_Final file is successfully saved as csv. and xlsx.')

In [23]:
# Save dataframe

#parsed_df.to_csv('Triples_Final.csv')
#parsed_df.to_excel(r"Triples_Final.xlsx", index=False)

In [ ]:
import pandas as pd

# Load the data from both Excel files
df1 = pd.read_excel('covid_neurodegeneration_triples.xlsx')
df2 = pd.read_excel('covid_neurodegeneration_triples_2run.xlsx')

# Step 1: Extract the relevant columns from both dataframes (Subject, Predicate, Object)
triples_df1 = df1[['Subject', 'Predicate', 'Object']]
triples_df2 = df2[['Subject', 'Predicate', 'Object']]

# Step 2: Remove duplicate triples in both dataframes
unique_triples_df1 = triples_df1.drop_duplicates()
unique_triples_df2 = triples_df2.drop_duplicates()

# Step 3: Merge the two dataframes to find common triples based on Subject, Predicate, and Object
common_triples = pd.merge(unique_triples_df1, unique_triples_df2, on=['Subject', 'Predicate', 'Object'])

# Step 4: Output the common triples
print("Common triples:")
print(common_triples)

# If you want to save the common triples to a file, uncomment the following line
# common_triples.to_excel('common_triples.xlsx', index=False)


# Separation into categories

In [ ]:
parsed_df

In [203]:
print(len(set_t))
set_t = set(parsed_df['Pathophysiological Process'])

1086


In [ ]:
set_t

In [208]:
categories_dict = {
    'Viral Entry and Neuroinvasion': {
        'ACE2': 3, 'TMPRSS2': 3, 'CD147': 3, 'receptor': 2, 'binding': 2, 
        'entry': 3, 'invasion': 3, 'spike_protein': 3, 'olfactory': 3, 
        'hematogenous': 3, 'blood-brain_barrier': 3, 'BBB': 3, 'neuroinvasion': 3, 
        'retrograde': 2, 'transport': 2, 'direct_infection': 3, 'direct_viral':2, 'neural': 2, 
        'neuron': 2, 'enteric_nervous_system': 3, 'route': 2, 'replication': 2, 
        'pathway': 1, 'transmission': 2, 'route':3
    },
    'Immune and Inflammatory Response': {
        'immune': 2, 'cytokine': 3, 'storm': 3, 'IL': 2, 'interleukin': 2, 
        'TNFa': 3, 'T_cell': 3, 'B_cell': 3, 'inflammatory': 3, 'inflammation': 3, 
        'microglia': 2, 'astrocyte': 2, 'immune_response': 3, 'Th1': 2, 
        'Th17': 2, 'immunity': 2, 'macrophage': 2, 'leukocyte': 2, 
        'neutrophil': 2, 'antibody': 3, 'complement': 2, 'infection':2, 'signaling':3, 'glial_cell':3
    },
    'Cellular and Molecular Neurodegenerative Mechanisms': {
        'amyloid': 3, 'tau': 3, 'synuclein': 3, 'prion': 3, 'aggregation': 2, 
        'misfolding': 2, 'oxidative_stress': 3, 'mitochondria': 3, 
        'neurodegeneration': 3, 'neurotoxicity': 3, 'neuronal_death': 3, 
        'apoptosis': 2, 'cell_death': 2, 'synapse': 2, 'excitotoxicity': 3, 
        'neurotransmitter': 2, 'synaptic_loss': 3, 'bioenergetic': 2, 
        'atrophy': 2, 'degradation': 2, 'dopamine':3, 'signaling':2, 'glia':2, 'grey_matter': 3, 
        'white_matter': 3, 'cortex': 3, 'posterior_fossa': 2, 'piriform_cortex': 2, 'hippocampus': 3, 
        'parahippocampal_gyrus': 3, 'amygdala': 2, 'basal_ganglia': 3, 'thalamus': 2, 'cerebellum': 2, 
        'prefrontal_cortex': 3, 'temporal_lobe': 3, 'frontal_lobe': 3, 'occipital_lobe': 2, 'parietal_lobe': 2, 
        'olfactory_bulb': 2, 'brainstem': 2, 'globus_pallidus': 2, 'substantia_nigra': 3
    },
    'Vascular-Related Neurodegenerative Effects': {
        'endothelial': 3, 'vascular': 3, 'BBB_dysfunction': 3, 'coagulation': 3, 
        'fibrin': 2, 'thrombus': 3, 'microclot': 3, 'platelet': 2, 
        'thrombosis': 3, 'stroke': 3, 'ischemia': 3, 'hypoperfusion': 2, 
        'oxygenation': 2, 'microhemorrhage': 3
    },
    'Psychological and Neurological Symptoms': {
        'depression': 3, 'depressive':3, 'anxiety': 3, 'cognitive': 3, 'memory': 3, 
        'brain_fog': 3, 'behavior': 2, 'mental': 2, 'psychological': 3, 
        'stress': 2, 'mood': 2, 'fatigue': 2, 'confusion': 2, 
        'neurological': 3, 'neurocognitive': 3, 'headache': 2, 
        'anosmia': 3, 'ageusia': 3, 'smell': 2, 'psychosis': 3, 
        'psychiatric': 3, 'neuromuscular': 2
    },
    'Systemic Cross-Organ Effects': {
        'gut': 3, 'lung': 3, 'systemic': 3, 'cross-organ': 3, 
        'multi-organ': 3, 'metabolic': 2, 'blood': 2, 'microbiota': 3, 
        'dysbiosis': 3, 'liver': 2, 'gastrointestinal': 3, 'pulmonary': 2, 
        'kidney': 2, 'endocrine': 2, 'hormonal': 2, 'hormone':2, 'comorbidity': 2, 
        'microbiome': 3, 'organ': 2, 'HPA_axis': 3, 'alveol':2
    }
}

In [209]:
from rapidfuzz import fuzz  # or from fuzzywuzzy import fuzz

# Set a similarity threshold for fuzzy matching
SIMILARITY_THRESHOLD = 80

# Initialize the group dictionary with each category as a key and an empty list as the value
group = {key: [] for key in categories_dict.keys()}
group['Uncategorized'] = []

# Loop over each item in the set to categorize it based on fuzzy matching
for item in set_t:
    # Convert the item to lowercase for case-insensitive matching
    item_lower = item.lower()
    
    # Initialize category scores with 0 for each category; this will store the accumulated score
    # for each category based on fuzzy matched keywords
    category_scores = {key: 0 for key in categories_dict}
    
    # Calculate the score for each category by summing the weights of matching keywords
    for category, keywords in categories_dict.items():
        for keyword, weight in keywords.items():
            # Convert the keyword to lowercase to match against the lowercase item text
            # Use fuzzy matching to check if the keyword is similar to any part of the item text
            similarity_score = fuzz.partial_ratio(keyword.lower(), item_lower)
            if similarity_score >= SIMILARITY_THRESHOLD:
                category_scores[category] += weight * (similarity_score / 100)  # Scale weight by similarity

    # Find the maximum score among all categories
    max_score = max(category_scores.values())
    
    #pat_process = item.split('@')[0]
    pat_process = item

    # Determine all categories that have the maximum score (if above zero)
    if max_score > 0:
        # Add item to each category that has the highest score
        for category, score in category_scores.items():
            if score == max_score:
                group[category].append(pat_process)
    else:
        # If no keywords matched, add item to 'Uncategorized'
        group['Uncategorized'].append(pat_process)

# `group` now contains the categorized items, allowing items to be in multiple categories if they
# have the same highest score across different categories


In [210]:
len(group['Uncategorized'])

170

In [ ]:
group['Uncategorized']

In [ ]:
reverse_mapping_group = {value: key for key, values in group.items() for value in values}

# Step 2: Map the values in 'existing_column' based on the reverse lookup
parsed_df['Group'] = parsed_df['Pathophysiological Process'].map(reverse_mapping_group)
parsed_df

In [215]:
#parsed_df.to_excel(r"Triples_Final_Grouped.xlsx", index=False)

## Convert dois to pmids

In [ ]:
import requests
import pandas as pd
import time
import re

# Load the dataset
file_path = "../output/covid_neurodegeneration_triples.csv"  # Update the path if needed
df = pd.read_csv(file_path)

# Function to clean DOI (remove "https://doi.org/" if present)
def clean_doi(doi):
    return re.sub(r"https?://doi\.org/", "", str(doi).strip())

# Function to convert DOI to PMID
def fetch_pmid_from_doi(doi):
    cleaned_doi = clean_doi(doi)
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={cleaned_doi}[doi]&retmode=json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        pmid = data["esearchresult"]["idlist"][0] if data["esearchresult"]["idlist"] else None
        return pmid
    except Exception as e:
        print(f"Error fetching PMID for DOI {cleaned_doi}: {e}")
        return None

# Apply DOI cleaning
df["DOI_Cleaned"] = df["DOI"].apply(clean_doi)

# Get unique cleaned DOIs
unique_dois = df["DOI_Cleaned"].dropna().unique()

# Map DOI to PMID
doi_to_pmid_mapping = {}
for doi in unique_dois:
    pmid = fetch_pmid_from_doi(doi)
    doi_to_pmid_mapping[doi] = pmid
    time.sleep(0.5)  # Respect NCBI API rate limits

# Add mapped PMIDs to dataframe
df["PMID"] = df["DOI_Cleaned"].map(doi_to_pmid_mapping)

# Save updated file
df.to_csv("covid_neurodegeneration_triples_with_pmids.csv", index=False)

# Display results
print(df.head())


## Fulltext triple extraction

In [ ]:
# Fulltext triple extraction on some test fulltexts
import json
import csv
import os
from openai import OpenAI

def gpt_authenticate():
    try:
        API_key = API_KEY  # Replace with your actual API key
        client = OpenAI(api_key=API_key)
        # Test the client with a simple API call
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hello"}],
            max_tokens=5
        )
        print("Authentication successful!")
        return client
        
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        return None

def gpt_extract(client, text):
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # or "gpt-4-turbo-preview"
            messages=[
                {
                    "role": "user",
                    "content": f'''Extract semantic triples (subject–predicate–object) from the paragraph below that describe the impact of COVID-19 on neurodegeneration:
                    Paragraph: "{text}"
                    
                    Example: 
                    Pathophysiological Process: Astrocyte_Activation
                    Triples:
                    SARS-CoV-2_infection|triggers|astrocyte_activation
                    
                    Follow this format strictly:
                    - Use ONLY the information from the paragraph.
                    - Replace spaces in names with underscores (_).
                    - Each triple must contain EXACTLY three elements separated by "|".
                    - Ensure "Pathophysiology Process:" and "Triples:" appear in the response.
                    '''
                }
            ],
            max_tokens=900,
            temperature=0.0  # Added for consistency
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in GPT extraction: {str(e)}")
        return ""

def process_files(dataset_dir, csv_output_path):
    client = gpt_authenticate()
    if not client:
        print("Failed to authenticate. Exiting...")
        return

    extracted_triples = []
    processed_count = 0

    try:
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

        for filename in os.listdir(dataset_dir):
            if filename.endswith(".json"):
                pmid = filename.split('.')[0]  # Remove .json extension
                print(f"\nProcessing file {processed_count + 1}: {filename}")
                file_path = os.path.join(dataset_dir, filename)
                
                try:
                    with open(file_path, "r", encoding="utf-8") as file:
                        data = json.load(file)
                        
                        if "paragraphs" in data:
                            for paragraph_idx, paragraph in enumerate(data["paragraphs"]):
                                if not paragraph or not paragraph.strip():
                                    continue
                                    
                                print(f"Processing paragraph {paragraph_idx + 1}")
                                extracted_text = gpt_extract(client, paragraph)
                                
                                if extracted_text:
                                    lines = extracted_text.split("\n")
                                    for line in lines:
                                        if "|" in line:
                                            parts = line.strip().split("|")
                                            if len(parts) == 3:
                                                subject, relation, obj = parts
                                                triple = [
                                                    pmid,
                                                    subject.strip(),
                                                    relation.strip(),
                                                    obj.strip()
                                                ]
                                                extracted_triples.append(triple)
                                                print(f"Extracted triple: {' | '.join(triple)}")
                
                except Exception as e:
                    print(f"Error processing file {filename}: {str(e)}")
                    continue
                
                processed_count += 1
                
                # Save progress periodically (every 5 files)
                if processed_count % 5 == 0:
                    save_to_csv(extracted_triples, csv_output_path)
                    print(f"\nProgress saved: {processed_count} files processed")

        # Final save
        save_to_csv(extracted_triples, csv_output_path)
        print(f"\nProcessing complete! Total files processed: {processed_count}")
        print(f"Total triples extracted: {len(extracted_triples)}")
        print(f"Results saved to: {csv_output_path}")

    except Exception as e:
        print(f"Fatal error in processing: {str(e)}")
        # Save what we have in case of fatal error
        save_to_csv(extracted_triples, csv_output_path)

def save_to_csv(triples, csv_path):
    try:
        with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(["PMID", "Subject", "Relation", "Object"])
            csv_writer.writerows(triples)
    except Exception as e:
        print(f"Error saving to CSV: {str(e)}")

if __name__ == "__main__":
    # Define file paths
    dataset_dir = "../data/test_fulltext/chunked_files"
    outputsave = "../data/test_fulltext"
    csv_output_path = os.path.join(outputsave, "extracted_triples.csv")
    
    # Run the extraction process
    process_files(dataset_dir, csv_output_path)

